In [188]:
import numpy as np 
import pandas as pd 
import json 
import os
import scipy
import collections

In [191]:
GameHistory = collections.namedtuple('GameHistory', 'player row col')

t = GameHistory(player=1, row=2, col=3)
t

GameHistory(player=1, row=2, col=3)

In [2]:
# os.chdir('c:\\Users\\Administrator\\OneDrive\\Desktop')
os.getcwd()

'c:\\Users\\Administrator\\OneDrive\\Desktop\\web-projects\\four-in-a-row-site'

In [3]:
with open('simulationResults.json') as file:
    raw_data = json.load(file)

# Display the data
# print(data)

In [4]:
def history_to_df(history):
    results = history['results']
    history = history['gameHistories']
    rows = []
    for game_index, game in enumerate(history):
        for move in game:
            move['game_index'] = game_index
            rows.append(move)


    df = pd.DataFrame(rows)
    df['result'] = df['game_index'].apply(lambda x: results[x])
    return df

In [6]:
game_history = history_to_df(raw_data)
game_history

,column,player,row,game_index,result
0,0,1,5,0,1
1,2,2,5,0,1
2,0,1,4,0,1
3,0,2,3,0,1
4,0,1,2,0,1
...,...,...,...,...,...
272,1,1,3,9,1
273,1,2,2,9,1
274,2,1,3,9,1
275,3,2,3,9,1


In [7]:
n_rows = 6
n_cols = 7

board = np.zeros((n_rows, n_cols))
board


array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.]])

In [8]:
# need to find a way to for the board to know which row a piece falls into
# given the column it is dropped into
def find_first_open_row(board, col):
    for row in range(board.shape[0] - 1, -1, -1):
        if board[row][col] == 0:
            return row

In [21]:

def find_legal_moves(board):
    legal_moves = []
    for col in range(board.shape[1]):
        if board[0][col] == 0:
            legal_moves.append(col)
    return legal_moves

def drop_piece(board, col, player):
    if col not in find_legal_moves(board):
        raise ValueError("Illegal move: column is full")
    row = find_first_open_row(board, col)
    board[row][col] = player
    return row, col


In [137]:
board = np.zeros((n_rows, n_cols))
drop_piece(board, 3, 1)
drop_piece(board, 3, 2)
drop_piece(board, 4, 1)
drop_piece(board, 3, 2)
drop_piece(board, 3, 1)
drop_piece(board, 3, 2)
drop_piece(board, 3, 1)
drop_piece(board, 2, 2)
drop_piece(board, 2, 1)
drop_piece(board, 1, 2)
drop_piece(board, 5, 1)
drop_piece(board, 0, 2)
# drop_piece(board, 6, 1)

for i in range(5):
    player = i % 2 + 1
    drop_piece(board, 0, player)

# drop_piece(board, 3, 2)
drop_piece(board, 6, 1)

for _ in range(3):
    drop_piece(board, 4, 1)
board

array([[1., 0., 0., 1., 0., 0., 0.],
       [2., 0., 0., 2., 0., 0., 0.],
       [1., 0., 0., 1., 1., 0., 0.],
       [2., 0., 0., 2., 1., 0., 0.],
       [1., 0., 1., 2., 1., 0., 0.],
       [2., 2., 2., 1., 1., 1., 1.]])

In [51]:
player = 1
board_slice = horizontal_slice 
kernel = np.ones(4) * player

print(board_slice)
print(kernel)
print(scipy.ndimage.convolve1d(board_slice, kernel, mode='constant', cval=np.nan))

# kernel

[2. 2. 2. 1. 1. 1. 1.]
[1. 1. 1. 1.]
[nan  7.  6.  5.  4. nan nan]
[  7.   6.   5.   4. 103. 202. 301.]


In [139]:
# Now need to determine if there is a winner
# Need to check if there are 4 in a row horizontally, vertically, or diagonally (positive slope or negative slope)
# Need to check if the last piece dropped is part of a 4 in a row
# def check_horizontal(board, row, col, player):
row = 5
col = 4
player = 1

horizontal_slice = board[row, max(col-3, 0):col+4]
vertical_slice = board[max(row-3, 0):row+4, col][::-1]
print(horizontal_slice)
print(vertical_slice)




[2. 2. 1. 1. 1. 1.]
[1. 1. 1. 1.]


In [140]:
# print(horizontal_slice)
# print(vertical_slice)

horizontal_indices = []
for i in range(-3, 4):
    if (col+i < 0) or (col+i >= board.shape[1]):
        continue
    else:
        horizontal_indices.append((row, col+i))
print(horizontal_indices)

vertical_indices = []
for i in range(-3, 4):
    if (row+i < 0) or (row+i >= board.shape[0]):
        continue
    else:
        vertical_indices.append((row+i, col))
print(vertical_indices)


negative_slope_slice = []
negative_slope_indices = []
for i in range(-3, 4):
    if (row+i < 0) or (col+i < 0) or (row+i >= board.shape[0]) or (col+i >= board.shape[1]):
        continue
    else:
        negative_slope_slice.append(board[row+i, col+i])
        negative_slope_indices.append((row+i, col+i))
print(negative_slope_slice)
print(negative_slope_indices)


positive_slope_slice = []
positive_slope_indices = []
for i in range(-3, 4):
    if (row-i < 0) or (col+i < 0) or (row-i >= board.shape[0]) or (col+i >= board.shape[1]):
        continue
    else:
        positive_slope_slice.append(board[row-i, col+i])
        positive_slope_indices.append((row-i, col+i))
print(positive_slope_slice)
print(positive_slope_indices)

[(5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6)]
[(2, 4), (3, 4), (4, 4), (5, 4)]
[0.0, 0.0, 2.0, 1.0]
[(2, 1), (3, 2), (4, 3), (5, 4)]
[1.0, 0.0, 0.0]
[(5, 4), (4, 5), (3, 6)]


In [154]:
input_indices = vertical_indices
# input_indices = vertical_indices
input_indices = negative_slope_indices
input_indices = positive_slope_indices

def _check_single_win_condition(board, input_indices, player):
    for i in range(len(input_indices) - 3):
        if (
            (board[input_indices[i][0], input_indices[i][1]] == player) and
            (board[input_indices[i+1][0], input_indices[i+1][1]] == player) and
            (board[input_indices[i+2][0], input_indices[i+2][1]] == player) and
            (board[input_indices[i+3][0], input_indices[i+3][1]] == player)
        ):
            print('winner!')
            return input_indices[i:i+4]
    return False
    # if np.all(board[i[0] + j, i[1] )

winning_indices = get_winning_indices(board, input_indices, player)
winning_indices

False

In [165]:
board = ConnectFourBoard()

board.make_move(0, 1)
print(board.board)
print(board.make_move(1, 1.0))
print(board.board)
print(board.make_move(2, 1))
print(board.board)
print(board.make_move(3, 1))
print(board.board)
print(board.game_state)
board.make_move(4, 1)

[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]]
None
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0. 0. 0.]]
None
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0.]]
None
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 0. 0. 0.]]
win


ValueError: Illegal move: game is over

In [104]:
input_slice = horizontal_slice

for i in range(len(input_slice) - 3):
    if np.all(input_slice[i:i+4] == player):
        print('winner')
        break

winner


In [89]:
player = 1
board_slice = horizontal_slice 
kernel = np.ones(4) * player

print(horizontal_slice)
print(scipy.ndimage.convolve1d(horizontal_slice, kernel, mode='constant', cval=np.nan))

print(vertical_slice)
print(scipy.ndimage.convolve1d(vertical_slice, kernel, mode='constant', cval=np.nan))

print(negative_slope_slice)
print(scipy.ndimage.convolve1d(negative_slope_slice, kernel, mode='constant', cval=np.nan))

print(positive_slope_slice)
print(scipy.ndimage.convolve1d(positive_slope_slice, kernel, mode='constant', cval=np.nan))

[2. 0. 0. 2. 0. 0.]
[nan  4.  2.  2. nan nan]
[0. 0. 0. 0. 1.]
[nan  0.  1. nan nan]
[0.0, 0.0, 1.0, 0.0, 0.0]
[nan  1.  1. nan nan]
[2.0, 0.0, 0.0, 1.0]
[nan  3. nan nan]


In [202]:
board = ConnectFourBoard()
player1 = ComputerPlayer(board)
player2 = ComputerPlayer(board)
game = ConnectFourGame(board, player1, player2)

game()

Game Over


In [203]:
print(game.board)
print(game.board.winner)
print(game.board.game_state)
print(game.board.winning_indices)
print(game.history.history)

[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 2. 1. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0.]
 [0. 0. 0. 2. 1. 0. 0.]
 [0. 2. 0. 2. 1. 1. 2.]]
1
win
[(2, 4), (3, 4), (4, 4), (5, 4)]
    player  row  col
0        1    5    4
1        2    5    3
2        1    4    4
3        2    4    3
4        1    5    5
5        2    5    6
6        1    3    4
7        2    5    1
8        1    3    3
9        2    2    3
10       1    2    4


# Build classes





In [196]:
class GameHistory:

    def __init__(self):
        self.player_number_history = []
        self.row_history = []
        self.col_history = []


    def record_move(self, player_number, row, col):
        self.player_number_history.append(player_number)
        self.row_history.append(row)
        self.col_history.append(col)

    def finalize_history(self):
        self.history = pd.DataFrame({
            'player': self.player_number_history,
            'row': self.row_history,
            'col': self.col_history
        })



In [201]:
class ConnectFourGame():
    
    def __init__(self, board, player1, player2):
        self.board = board
        self.history = GameHistory()
        self.player1 = player1
        self.player2 = player2
        self.current_player = self.player1
        self.current_player_number = 1

    def __call__(self):
        if self.board.game_state == 'undecided':
            move = self.current_player.get_move()
            try:
                row, col = self.board.drop_piece(move, self.current_player_number)
            except ValueError:
                print('Illegal Move: try again')
                self.play()
            self.history.record_move(self.current_player_number, row, col)
            if self.current_player == self.player1:
                self.current_player = self.player2
                self.current_player_number = 2
            else:
                self.current_player = self.player1
                self.current_player_number = 1
            self.__call__()   
        else:
            self.history.finalize_history()
            print('Game Over')    


In [194]:
class ConnectFourBoard():

    def __init__(self, n_rows=6, n_cols=7):
        self.n_rows = n_rows
        self.n_cols = n_cols
        self.board = np.zeros((n_rows, n_cols))
        self.legal_moves = self.get_legal_moves()
        self.game_state = 'undecided'
        self.winning_indices = None
        self.winner = None


    def drop_piece(self, col, player_number):
        if col not in self.legal_moves:
            raise ValueError("Illegal move: column is full or does not exist")
        if self.game_state != 'undecided':
            raise ValueError("Illegal move: game is over")
        if player_number not in [1, 2]:
            raise ValueError("Illegal move: player must be 1 or 2")
        row = self._get_first_open_row(col)
        self.board[row][col] = player_number
        self.legal_moves = self.get_legal_moves()
        if winning_indices := self.check_win(row, col, player_number):
            self.game_state = 'win'
            self.winning_indices = winning_indices
            self.winner = player_number
        elif len(self.legal_moves) == 0:
            self.game_state = 'tie'
        return row, col
        
    def get_legal_moves(self):
        legal_moves = []
        for col in range(self.board.shape[1]):
            if self.board[0][col] == 0:
                legal_moves.append(col)
        return legal_moves
    
    def check_win(self, row, col, player):
        horizontal_indices = self._get_horizontal_indices(row, col)
        vertical_indices = self._get_vertical_indices(row, col)
        negative_slope_indices = self._get_negative_slope_indices(row, col)
        positive_slope_indices = self._get_positive_slope_indices(row, col)
        input_indices = [horizontal_indices, vertical_indices, negative_slope_indices, positive_slope_indices]
        for indices in input_indices:
            if winning_indices := self._check_single_win_condition(indices, player):
                return winning_indices
        return False

    def _get_first_open_row(self, col):
        for row in range(self.board.shape[0] - 1, -1, -1):
            if self.board[row][col] == 0:
                return row
            
    def _check_single_win_condition(self, input_indices, player_number):
        for i in range(len(input_indices) - 3):
            if (
                (self.board[input_indices[i][0], input_indices[i][1]] == player_number) and
                (self.board[input_indices[i+1][0], input_indices[i+1][1]] == player_number) and
                (self.board[input_indices[i+2][0], input_indices[i+2][1]] == player_number) and
                (self.board[input_indices[i+3][0], input_indices[i+3][1]] == player_number)
            ):
                return input_indices[i:i+4]
        return False
            
    def _get_horizontal_indices(self, row, col):
        indices = []
        for i in range(-3, 4):
            if (col+i < 0) or (col+i >= self.board.shape[1]):
                continue
            else:
                indices.append((row, col+i))
        return indices
    
    def _get_vertical_indices(self, row, col):
        indices = []
        for i in range(-3, 4):
            if (row+i < 0) or (row+i >= self.board.shape[0]):
                continue
            else:
                indices.append((row+i, col))
        return indices
    
    def _get_negative_slope_indices(self, row, col):
        indices = []
        for i in range(-3, 4):
            if (row+i < 0) or (col+i < 0) or (row+i >= self.board.shape[0]) or (col+i >= self.board.shape[1]):
                continue
            else:
                indices.append((row+i, col+i))
        return indices
    
    def _get_positive_slope_indices(self, row, col):
        indices = []
        for i in range(-3, 4):
            if (row-i < 0) or (col+i < 0) or (row-i >= self.board.shape[0]) or (col+i >= self.board.shape[1]):
                continue
            else:
                indices.append((row-i, col+i))
        return indices
    
    def __str__(self):
        return str(self.board)
    
    def __repr__(self):
        return str(self.board)

In [195]:
class ComputerPlayer():
    def __init__(self, board):
        self.board = board

    def get_move(self):
        return np.random.choice(self.board.legal_moves)

# Scatch code below this

In [43]:
?scipy.ndimage.convolve1d

Signature:
scipy.ndimage.convolve1d(
    input,
    weights,
    axis=-1,
    output=None,
    mode='reflect',
    cval=0.0,
    origin=0,
)
Docstring:
Calculate a 1-D convolution along the given axis.

The lines of the array along the given axis are convolved with the
given weights.

Parameters
----------
input : array_like
    The input array.
weights : ndarray
    1-D sequence of numbers.
axis : int, optional
    The axis of `input` along which to calculate. Default is -1.
output : array or dtype, optional
    The array in which to place the output, or the dtype of the
    returned array. By default an array of the same dtype as input
    will be created.
mode : {'reflect', 'constant', 'nearest', 'mirror', 'wrap'}, optional
    The `mode` parameter determines how the input array is extended
    beyond its boundaries. Default is 'reflect'. Behavior for each valid
    value is as follows:

    'reflect' (`d c b a | a b c d | d c b a`)
        The input is extended by reflecting about 

In [150]:
board

array([[1., 0., 0., 1., 0., 0., 0.],
       [2., 0., 0., 2., 0., 0., 0.],
       [1., 0., 0., 1., 1., 0., 0.],
       [2., 0., 0., 2., 1., 0., 0.],
       [1., 0., 1., 2., 1., 0., 0.],
       [2., 2., 2., 1., 1., 1., 1.]])

In [14]:
len(game_histories)

10

In [128]:
list(range(len(input_indices) - 3))

[0]

In [130]:
for i in range(len(input_indices) - 3):
    print(i)

0


In [131]:
len(input_indices)

4

In [147]:
input_indices

[(5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6)]

In [143]:
board[winning_indices[0]]

2.0

In [144]:
input_indices[i:i+4][0]

(5, 2)